In [5]:
from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
import numpy as np
from keras.preprocessing import image

In [2]:
'''
   Definição da rede neural convolucional

    * Criação da cnn no modelo sequencial (sequencia de layers)
    * Criação de duas camadas de convolução com função de ativação Relu, 
      seguidas pelos processos de normalização dos mapas de características e max Pooling.
      O processo de flattening é adicionado ao final das camadas. 
'''

classifier = Sequential()

classifier.add(Conv2D(32, (3,3), input_shape = (64, 64, 3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Conv2D(32, (3,3), activation = 'relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size = (2,2)))

classifier.add(Flatten())

'''
    * Adição das hidden Layers
    * Aplicação da operação de dropout às saídas 
        Busca zerar uma determinada quantidade de entradas 
        a fim de otimizar o sistema e reduzir o overfitting
    * Adição da camada de saída, utilizando a sigmóide como função de ativação
        A sigmóide é utilzada em classificações binárias
        OBS -> units = 1 indica que há apenas uma unidade de saída para
        a classificação binária
'''
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 1, activation = 'softmax'))

'''
    * Compilação da rede neural
        OBS: Para a classificação binária, a função de perda será definida 
        por 'binary_crossentropy'. Para a classificação em múltiplas classes
        é necessário utilizar o 'categorical_crossentropy'
'''

classifier.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])


In [3]:
'''
    Geração das novas imagens, pelo processo de augmentation
    generatedTrain e generatedTest contém as definições 
    de como as imagens serão modificadas pelo Image data Generator 
    
    OBS: O parâmetro 'rescale' do ImageDataGenerator realiza o 
    processo de normalização de modo automático
'''
generatedTrain = ImageDataGenerator(rescale = 1./255,
                                    rotation_range = 7,
                                    horizontal_flip = True,
                                    shear_range = 0.2,
                                    height_shift_range = 0.07,
                                    zoom_range = 0.2)

generatedTest = ImageDataGenerator(rescale = 1./255)

'''
    A base de dados para o treinamento é criada, utilizando o método flow_from_directory nas 
    variáveis generatedTrain e generatedTest a fim de buscar o dataset armazenado em um determinado 
    diretório e aplicar as definições descritas previamentes pela classe ImageDataGenerator.

    * Descrição dos parâmetros utilizados no método flow_from_directory:
    * 'dataset_catDog/training_set' -> Descreve o caminho para o dataset 
    * target_size -> Descreve o tamanho das imagens para entrada
    * batch_size -> Define o número de amostras que serão propagadas pela rede
    * class_mode -> Define a forma de classificação (As classes são definidas pelas pastas presentes no dataset)
'''

trainDatabase = generatedTrain.flow_from_directory('dataset_catDog/training_set', 
                                                   target_size = (64,64), 
                                                   batch_size = 32, 
                                                   class_mode = 'binary')

testDatabase = generatedTest.flow_from_directory('dataset_catDog/test_set', 
                                                   target_size = (64,64), 
                                                   batch_size = 32, 
                                                   class_mode = 'binary')

Found 4000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [4]:
''' 
    Etapa de treinamento da rede neural. É importante que a função 'fit_generator' está sendo
    utilizada ao invés da função 'fit' uma vez que ela suporta o processo de augmentation, contudo
    tal função está em processo de depreciação, uma vez que em versões mais atuais, a função 'fit' 
    também suporta.

    Explicação dos parâmetros:

    * trainDatabase -> Dados para treino (após a augmentation)
    * steps_per_epoch -> Número total de etapas (lotes de amostras) a serem produzidas 
                         pelo gerador antes de declarar uma época concluída e iniciar a próxima época.
                         É importante citar que o valor ideal para este parâmetro se dá pela quantidade 
                         total de imagens para treinamento (caso haja um alto poder de processamento) ou
                         pelo total de amostras dividido pel valor do batch_size (caso haja um baixo 
                         poder de processamento)
    * epochs -> Epocas de treinamento da rede
    * validation_data -> Dados para a validação (após a augmentation)
    * validation_steps -> Possui o mesmo princípio do 'steps_per_epoch', porém levando em 
                          consideração a etapa de validação. O valor ideal para este parâmetro 
                          se dá pelo total de amostras dividido pel valor do batch_size
'''

classifier.fit_generator(trainDatabase, 
                         steps_per_epoch = 4000/32, 
                         epochs = 5, 
                         validation_data = testDatabase,
                         validation_steps = 1000/32)

C:\Users\marci\AppData\Local\Temp\ipykernel_13972\1269131039.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(trainDatabase,


Epoch 1/5


125/125 [==============================] - 38s 290ms/step - loss: 0.7914 - accuracy: 0.5000 - val_loss: 0.8050 - val_accuracy: 0.5000
Epoch 2/5
125/125 [==============================] - 12s 95ms/step - loss: 0.6596 - accuracy: 0.5000 - val_loss: 0.6789 - val_accuracy: 0.5000
Epoch 3/5
125/125 [==============================] - 12s 95ms/step - loss: 0.6264 - accuracy: 0.5000 - val_loss: 0.7353 - val_accuracy: 0.5000
Epoch 4/5
125/125 [==============================] - 12s 94ms/step - loss: 0.5995 - accuracy: 0.5000 - val_loss: 0.6063 - val_accuracy: 0.5000
Epoch 5/5
125/125 [==============================] - 12s 97ms/step - loss: 0.5736 - accuracy: 0.5000 - val_loss: 0.5903 - val_accuracy: 0.5000


In [11]:
'''
    Previsões feitas com uma determinada imagem
    Uma imagem é passada como entrada e o retorno será a sua classificação

    Explicação dos processos:
    
    * image.load_img() -> A imagem para previsão é carregada no sistema e o valor de suas dimensões é definido
    * image.img_to_array() -> A imagem tem o seu tipo alterado para um Array
    * testImage /= 255 -> Os valores do array são normalizados
    * np.expand_dims() -> Há uma expansão das dimensões da imagem
        dimensões originais - (64,64,3) -> largura, altura, canais
        dimensões transformadas - (1,64,64,3) -> batch, largura, altura, canais
    * classifier.predict() -> Realiza a previsão, o retorno sera um float entre 0 e 1, indicando a 
        probabilidade de pertencer à classe 0 ou à classe 0 (Essa estrutura advém da sigmóide)
    * trainDatabase.class_indices -> Exibe qual valor está atribuído a qual classe, neste caso temos:
        0 - cachorro
        1 - gato

    * OBS -> Neste caso, a imagem carregada será a de uma gato
'''
testImage = image.load_img('dataset_catDog/test_set/gato/cat.3513.jpg', target_size = (64,64))
testImage = image.img_to_array(testImage)
testImage /= 255
testImage = np.expand_dims(testImage, axis = 0)

prev = classifier.predict(testImage)
print(prev)
print(trainDatabase.class_indices)

1/1 [==============================] - 0s 25ms/step
[[1.]]
{'cachorro': 0, 'gato': 1}
